In [1]:
from gcpspark import *
import pyspark.sql.functions as F
from pyspark import StorageLevel
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import *
import os

Dotenv installed
Loaded variables .env True
Pyspark installed
JAVA:  /usr/lib/jvm/temurin-8-jdk-amd64
DATAPROC:  True
Current GCP Project Name: cencosudx
Current ENVIRONMENT: staging
<class 'Exception'>


In [2]:
spark = create_pyspark(name="co-contribucion3" , 
                       connection="gcp")

File already exists at: /jars/spark-bigquery-with-dependencies_2.12-0.26.0.jar
File already exists at: /jars/gcs-connector-hadoop3-2.2.19.jar
Process: co-contribucion3_1746117431367


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/01 16:37:15 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
25/05/01 16:37:15 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
25/05/01 16:37:15 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
25/05/01 16:37:15 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [3]:
df = spark.read.parquet("gs://staging-da-contribution/CO/sales")

In [4]:
df.printSchema()

root
 |-- EAN: string (nullable = true)
 |-- Item_Id: string (nullable = true)
 |-- Location_Id: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Hour_Sales: string (nullable = true)
 |-- Sales_Channel: string (nullable = true)
 |-- Quantity_Sales: double (nullable = true)
 |-- NetAmount: double (nullable = true)
 |-- NetCost: double (nullable = true)



In [5]:
df = df.filter(F.col("Date")>=datetime(2024,1,1))

In [6]:
df=df.filter(~F.isnull("NetAmount"))

In [7]:
df=df.filter(F.col("NetAmount")!=0)

In [8]:
SalesChannel = ['5','6','7']

In [9]:
df = df.filter(~df['Sales_Channel'].isin(SalesChannel))
df = df.withColumn("Hour_Sales", F.col("Hour_Sales").substr(1, 2).cast(IntegerType()))

In [10]:
df = df.withColumn("Contribution", F.round(F.col("NetAmount") - F.col("NetCost")))
df = df.withColumn("Margen",F.col("Contribution") / F.col("NetAmount"))

In [11]:
# df.cache()

In [12]:
df.write.mode("overwrite").parquet("gs://staging-da-contribution/CO/final_sales")

In [13]:
print("Done")

Done


In [14]:
spark.stop()

In [ ]:
df_1 = df
#df_1 = df_1.withColumn("updated_at", F.to_date("updated_at"))
df_sku_por_mes = df_1.groupBy(F.year("Date").alias("year"), F.month("Date").alias("month")).agg(F.countDistinct("Item_Id").alias("cant_sku"))
df_sku_por_mes = df_sku_por_mes.orderBy("year", "month")
df_sku_por_mes.show(50)